# En el Notebook TFM_clientes5 creamos 2 dataframe :
     # El primero es la base de nuestros datos ya tratados:
           df_fecha.to_csv('data/df_tfm_base.csv',sep='|',index=False)
        
     # El segundo es sóló la parte de importe2 con las variables de importe de meses anteriores por cliente
         df_cl_tot.to_csv('data/df_cl_tot.csv',sep='|',index=False)

# En el Notebook TFM_clientes6 Creamos un autoarima para los clientes estacionales, donde predecimos 12 meses por cliente
       # Exportamos los estacionales y no estacionales para trabajarlos aparte y buscar clusters
            df_SI_est.to_csv('data/df_SI_est.csv',sep='|',index=False)
            df_NO_est.to_csv('data/df_NO_est.csv',sep='|',index=False)
   
       # Fichero total diferenciando estacionalida yno estacionalidad
          df_est_total.to_csv('data/df_est_total.csv',sep='|',index=False)
       
       # Exportamos las predicciones
            df_tot_exito0.to_csv('data/Prediccion.csv',sep='|',index=False)
       

# Vamos ahora a repetir el ejercicio pero prediciendo los 2 meses siguientes, aunque nos quedemos con el segundo, y repitendo mes a mes

# En el progrma clientes8_1 y 8_2 pero desglosamos el 7 en 2 veces porque excede de memoria: hasta el cliente 4000 y a partir del 4000

# Repetimos para NO ESTACIONALES




In [ ]:
# Librerias a usar
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy import stats
from sklearn.tree import DecisionTreeRegressor

import datetime
from datetime import datetime
from fbprophet import Prophet

import statsmodels.graphics.tsaplots as sgt 
import statsmodels.tsa.stattools as sts 
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
sns.set()

from statsmodels.tsa.arima_model import ARMA
from scipy.stats.distributions import chi2
from statsmodels.tsa.ar_model import AR,ARResults
from statsmodels.tsa.arima.model import ARIMA


from statsmodels.compat.pandas import (
    Appender,
    Substitution,
    call_cached_func,
    to_numpy,
)
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

from statsmodels.tsa.stattools import acf

from numpy import log

from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA

from random import random
import os

import pmdarima as pm
from pmdarima.utils import tsdisplay
from pmdarima.preprocessing import BoxCoxEndogTransformer
from pmdarima.pipeline import Pipeline


In [ ]:

from __future__ import annotations


In [ ]:
# Recuperamos los fciheros anteriores de estacionalidad y no estacionalidad que son con los que vamos a trabajar
    # ya no hace falta tener fecha como indice, pero si como columna

df_NO_est=pd.read_csv('data/df_NO_est.csv',sep='|')

# df_NO_est=pd.read_csv('data/df_NO_est.csv',sep='|')
# df_est_total=pd.read_csv('data/df_est_total.csv',sep='|')

df_NO_est.head()


In [ ]:
import pandas
from statsmodels.tsa.arima_model import ARIMA

import pmdarima as pm


In [ ]:
 # Nos quedamos con cliente, fecha e importe y llevamos la Fecha a indice
df_fecha_si=df_NO_est.loc[:,['id_cl','Fecha','importe2']]   
df_fecha_si.Fecha = pd.to_datetime(df_fecha_si.Fecha)
df_fecha_si=df_fecha_si.set_index('Fecha')  # convertimos la fecha a indice
df_fecha_si.head()

# modelo autoarima mes a mes

In [ ]:
# Cremos las funciones necesarias
    
    # creamos una funcion que nos calcula el valor de hace 2 meses, la diferencia y el incremento
def create_vars(df):
    data = df.copy()
        
    data['imp_2m'] = data['importe2'].shift(2)
    data['dif_imp2']=data['importe2']-data['imp_2m']
    data['incr_imp2']= (data['dif_imp2']/data['imp_2m']).fillna(0)

  #  data['y_2m'] = data['Y'].shift(2)
  #  data['dif_y2m']=data['Y']-data['y_2m']
  #  data['incr_y2']= (data['dif_y2m']/data['y_2m']).fillna(0)
    
    data['dif_y2m']=data['Y']-data['imp_2m']
    data['incr_y2']= (data['dif_y2m']/data['imp_2m']).fillna(0)
    
    return data

    # calculamos si la tendencia entre 2 meses es positiva o negativa en los datos reales y los predichos
    # nuestro exito sera si lsa 2 tendencias son iguales
def create_tend(df):
    data = df.copy()
    
    data['tend_imp2']= np.where(data['dif_imp2']<=0, -1, 1)
    data['tend_y2m']= np.where(data['dif_y2m']<=0, -1, 1)
    data['exito']=np.where((data['tend_imp2']-data['tend_y2m'])==0,1,0)  
    return data


In [ ]:


lista_dfs_cl_tot = []
counter = 1

for cliente, df_cl_pred in list(df_fecha_si.groupby('id_cl'))[10000:14000]:
    try:
        print(counter)
        counter += 1
        X=df_cl_pred    

            # creamos un indicador de periodo para trabajar mas comodamente, le pedimos uno mas porque la lista empieza por 0
        i =list(range(37))
        X['periodo']=i[1:]
        
        lista_cli12 = []
            # creamos un rango de 12 meses que son las predicciones que vamos a hacer
        imes=1
        for j in range(0,11):
            print(imes)
            imes+=1
                # Dejamos 2 meses para test
            X_train = X.loc[X.periodo<=24+j]
            y_test = X.loc[(X.periodo>(24+j)) & (X.periodo<=(24+j+2))]

                # creamos el modelo con un autoarima
            model = pm.auto_arima(X_train.importe2, start_p=1, start_q=1,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=4, max_q=3, # maximum p and q
                          m=1,              # frequency of series
                          d=None,           # let model determine 'd'
                          seasonal=False,   # No Seasonality
                          start_P=0, 
                          alpha=0.05,
                          D=0, 
                          trace=True,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True)
  
            model.fit(X_train.importe2)
                # creamos la prediccion
            n_periods = 2
            fitted, confint = model.predict(n_periods=n_periods, return_conf_int=True)

                # Make as pandas series
            fitted_series = pd.Series(fitted, index=y_test.index)
            lower_series = pd.Series(confint[:, 0], index=y_test.index)
            upper_series = pd.Series(confint[:, 1], index=y_test.index)
            
            print('fitted',fitted_series)
            
            if lista_cli12:
                lista_cli12.append(fitted[1])
            else:
                lista_cli12.append(fitted[0])
                lista_cli12.append(fitted[1])
            
            
            print('lista pred:', lista_cli12)
            
        # lo llevamos a un dataframe para poderlo unir a nuestro dataframe y comparar 
        # le añadimos el periodo para unir por eses campo
        lista_cli12_b=np.array(lista_cli12)
        data_fit=pd.DataFrame(lista_cli12_b)
        data_fit['periodo']=i[25:]
        print(data_fit)
        
            #cambiamos el nombre a la prediccion
        data_fit=data_fit.rename({0: 'Y'}, axis=1)
            # y unimos a los datos reales. Sólo la prediccion para comparar
        X_fin=pd.merge(X,data_fit,left_on=['periodo'], right_on=['periodo'],how='left').loc[23:]

        print(X_fin)
            
            # recuperamos el valor real del mes 24 para pegarselos a las predicciones y poder calcular diferencias a 2 meses# 
        #val24=X_fin.importe2[23]
       # X_fin['Y']= X_fin['Y'].replace(np.nan,val24)
        
       # print(X_fin)

            # Creamos variables nuevas para ver si el resultado es bueno
        Mod_df0=create_vars(X_fin)
            
        print(Mod_df0)
            
        Mod_df=create_tend(Mod_df0).loc[24:] 
            
        print(Mod_df)
        lista_dfs_cl_tot.append(Mod_df)  # solo sacamos la comparativa de las predicciones
                                          # el exito empezara a contar en el periodo 26
    except:
        print(counter, "ha fallado el modelo")
        continue

In [ ]:
lista_dfs_cl_tot[2]

In [ ]:
len(lista_dfs_cl_tot)

In [ ]:
df_tot_exito0=pd.concat(lista_dfs_cl_tot)
df_tot_exito=pd.DataFrame(df_tot_exito0.groupby('id_cl')['exito'].sum())
df_tot_exito

In [ ]:
df_tot_exito=df_tot_exito.reset_index()
n_tot_exito=df_tot_exito.groupby(['exito'])['id_cl'].count()
n_tot_exito

In [ ]:
# unimos ambos fichero
df_No_est_exito=pd.merge(df_tot_exito0,df_tot_exito.drop(['exito'], axis=1), left_on=['id_cl'], right_on=['id_cl'],how='left')
df_No_est_exito.head(12)"

In [ ]:
# Exportamos las predicciones
   # v1: < 4000\n",
   # v2: 4000-10000\n",
 # v3: 10000-14000
df_No_est_exito.to_csv('data/Prediccion_No_est_v3.csv',sep='|',index=False)
                